In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws

In [2]:
import cloudknot as ck

In [3]:
def download_and_track(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.utils as ut
    import diff_classifier.aws as aws
    import os.path as op
    import pandas as pd
    
    aws.download_s3(filename, op.split(filename)[1])
    
    outfile = 'Traj_' + op.split(filename)[1].split('.')[0] + '.csv'
    local_im = op.join(os.getcwd(), op.split(filename)[1])
    if not op.isfile(outfile):
        ij.track(local_im, outfile, template=None, fiji_bin=None, radius=4.5, threshold=0., 
              do_median_filtering=True, quality=4.5, median_intensity=300.0, snr=0.0, 
              linking_max_distance=7.0, gap_closing_max_distance=10.0, max_frame_gap=2,
              track_displacement=10.0)

        aws.upload_s3(outfile, op.join(op.split(filename)[0], outfile))
    print("Done with tracking.  Should output file of name {}".format(op.join(op.split(filename)[0], outfile)))
    return ut.csv_to_pd(outfile)

In [ ]:
# for i in range(0, 4):
#     for j in range(0, 4):

#         download_and_track('01_18_Experiment/P1/P1_S1_L_0000_{}_{}.tif'.format(i, j))

In [4]:
to_track = []
for i in range(0, 4):
    for j in range(0, 4):
        to_track.append('01_18_Experiment/P1/P1_S1_L_0000_{}_{}.tif'.format(i, j))

In [ ]:
to_track

In [5]:
github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
my_image = ck.DockerImage(func=download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

#Step here: delete user info from newly created docker image.

In [6]:
knot = ck.Knot(name='download_and_track_with_my_image_5',
               docker_image = my_image,
               pars_policies=('AmazonS3FullAccess',))

In [7]:
results = knot.map(to_track)

In [ ]:
knot.clobber

In [19]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
0f60201c-75bf-4c2c-9ae5-565d67a756e6        download_and_track_with_my_image_5-6        RUNNABLE 
c8de7840-2bd5-44b1-b5eb-bc370873aef0        download_and_track_with_my_image_5-8        RUNNABLE 
d280153c-fcf2-46cf-a627-ccf3aefb63af        download_and_track_with_my_image_5-2        RUNNABLE 
4ff2637b-b602-4265-8fdf-ed5a1eb182c0        download_and_track_with_my_image_5-13        RUNNABLE 
772df832-8a9d-4718-9c09-409aa53dfc72        download_and_track_with_my_image_5-11        RUNNABLE 
636c6c81-d2ef-4027-8e3c-40bbf381e558        download_and_track_with_my_image_5-7        RUNNABLE 
7dfd4cbf-9b76-4482-b8f9-86cf2b7d78ce        download_and_track_with_my_image_5-15        RUNNABLE 
73e419a8-3564-4fb7-85c8-dd7d49e76fec        download_and_track_with_my_image_5-10        RUNNABLE 
3a43da2b-ed2f-409b-abaf-d83fe38b3204        download_and_track_with_my_image_5-4        RUNNABLE

In [21]:
results

[<Future at 0x7fd6a9dba8d0 state=finished raised ValueError>,
 <Future at 0x7fd6a9f1dc50 state=finished raised ValueError>,
 <Future at 0x7fd6a9ed4a90 state=running>,
 <Future at 0x7fd6a98ef190 state=finished raised ValueError>,
 <Future at 0x7fd6a999f8d0 state=running>,
 <Future at 0x7fd6a8446390 state=running>,
 <Future at 0x7fd6a8446310 state=running>,
 <Future at 0x7fd6a8462150 state=running>,
 <Future at 0x7fd6a8462750 state=running>,
 <Future at 0x7fd6a98ef090 state=running>,
 <Future at 0x7fd6a858fed0 state=running>,
 <Future at 0x7fd6a85bcdd0 state=running>,
 <Future at 0x7fd6a8d937d0 state=running>,
 <Future at 0x7fd6a8d933d0 state=running>,
 <Future at 0x7fd6a8d93e50 state=running>,
 <Future at 0x7fd6a85524d0 state=running>]

In [22]:
r0 = results[0]
r0.result()

ValueError: unsupported pickle protocol: 4

In [ ]:
def download_and_split(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    local_name = op.split(filename)[1]
    DIR = op.split(filename)[0]
    try1 = filename.split('.')[0] + '_0_0.tif'
    try2 = filename.split('.')[0] + '_3_3.tif'
    
    s3 = boto3.client('s3')
    try:
        obj = s3.head_object(Bucket='ccurtis7.pup', Key=try1)
    except:
        try:
            obj = s3.head_object(Bucket='ccurtis7.pup', Key=try2)
        except:
            aws.download_s3(filename, local_name)
            names = ij.partition_im(local_name)
            for name in names:
                aws.upload_s3(name, op.join(op.split(filename)[0], name))

In [ ]:
pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
hemis = ["R", "L"]
videos = 15
folder = '01_18_Experiment'

In [ ]:
missing = []
for i in range(10, 15):
    missing.append("P1_S2_R_00{}.tif".format(i))

for i in range(10, 15):
    missing.append("P2_S3_L_00{}.tif".format(i))
    
for i in range(0, 15):
    missing.append("P3_S3_L_{}.tif".format("%04d" % i))

In [ ]:
to_partition = []
for pup in pups:
    for slic in slices:
        for hemi in hemis:
            for i in range(0, videos):
                name = "{}/{}/{}_{}_{}_{}.tif".format(folder, pup, pup, slic, hemi, "%04d" % i)
                if not name in missing:
                    to_partition.append(name)

In [ ]:
for name in to_partition:
    download_and_split(name)

In [ ]:
!rm P1_S1_R_0000*

In [ ]:
to_partition

In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3')

try:
    s3.Object("ccurtis7.pup", '01_18_Experiment/test_file.tif').load()

In [ ]:
s3 = boto3.client('s3')

result = s3.list_objects_v2(Bucket="ccurtis7.pup", Prefix="/01_18_Experiment/test_filebook.tif")

exists = False
if result:
    exists = True

print(exists)

In [ ]:
for obj in result.get('Contents', []):
    if obj['Key'] == "/01_18_Experiment/test_filebook.tif":
        print('Object here')
    else:
        print('Object not here')

In [ ]:
try:
    obj = s3.head_object(Bucket='ccurtis7.pup', Key="01_18_Experiment/test_file.tif")
except:
    print("Does not exist")